In [1]:
import pandas as pd
import numpy as np

In [2]:
from import_data import facultyData
minorityReserveB, minorityReserveDA, M, shiteiDict, fac, bunri, categoryToList=facultyData.main()
fill_data=dict()
for f in fac:
    fill_data[f]=[0, 0] #mr, mq
for i in range(1000):
    fill_mir_df = pd.read_csv(f"MultiThread/fill_rate_mir{i}.csv", header=0, index_col=0)
    fill_Maq_df = pd.read_csv(f"MultiThread/fill_rate_Maq{i}.csv", header=0, index_col=0)
    for f in fac:
        fill_data[f][0] += fill_mir_df.loc[f, "fill rate"]
        fill_data[f][1] += fill_Maq_df.loc[f, "fill rate"]
        
for f in fac:
    fill_data[f][0] = fill_data[f][0]/1000
    fill_data[f][1] = fill_data[f][1]/1000
        
df = pd.DataFrame(list(fill_data.values()), columns=["MRの定員充足率", "MQの定員充足率"], index=list(fill_data.keys()))
print(df)
df.to_csv("fill_rate.csv", encoding="utf-8-sig")

               MRの定員充足率  MQの定員充足率
法学部            0.913984  0.950654
経済学部           0.895991  0.915804
A群（思想文化）       0.998381  0.824952
B群（歴史文化_日本史学）  0.994091  0.562909
C群（歴史文化_東洋史学）  0.775053  0.322474
...                 ...       ...
地球惑星環境学        1.000000  1.000000
化学             0.999615  1.000000
生物化学           1.000000  1.000000
生物情報科学         1.000000  1.000000
生物学            0.998200  0.999700

[78 rows x 2 columns]


In [4]:
utility_df = pd.read_csv("Result/utilitySum.csv", header=0)
fill_rate_df = pd.read_csv("fill_rate.csv", header=0)
columns=utility_df.columns.to_list()
columns[0]="学部"
utility_df.columns=columns
columns=fill_rate_df.columns.to_list()
columns[0]="学部"
fill_rate_df.columns=columns

In [5]:
df = pd.merge(utility_df, fill_rate_df)

In [6]:
columns=df.columns.to_list()
columns[0]="学部"
df.columns=columns

In [8]:
print(df)

               学部  指定科類枠の効用増減  全科類枠の効用増減   全体での効用増減  MRの定員充足率  MQの定員充足率
0             法学部   -1.765251  -4.174694  -5.939945  0.913984  0.950654
1            経済学部  -13.503607   6.747148  -6.756460  0.895991  0.915804
2        A群（思想文化）    0.000000   6.451483   6.451483  0.998381  0.824952
3   B群（歴史文化_日本史学）    0.000000  34.747199  34.747199  0.994091  0.562909
4   C群（歴史文化_東洋史学）    0.000000   7.250555   7.250555  0.775053  0.322474
..            ...         ...        ...        ...       ...       ...
73        地球惑星環境学    0.000000   0.000000   0.000000  1.000000  1.000000
74             化学   -0.444989   0.000000  -0.444989  0.999615  1.000000
75           生物化学   -0.676768   0.000000  -0.676768  1.000000  1.000000
76         生物情報科学   -0.359216   0.000000  -0.359216  1.000000  1.000000
77            生物学    0.000000  -0.120055  -0.120055  0.998200  0.999700

[78 rows x 6 columns]


In [10]:
df.to_csv("fill_and_utility.csv", encoding="utf-8-sig", index=0)

In [11]:
i=0
submit = pd.read_csv(f"MultiThread/created_submit_komaba{i}.csv", header=0)
k="l2"
fac="経済学部"
df=submit[submit["karui"]==k]
row, col=df.shape
cnt_rank=0
cnt_number=0
length_pref=0
for r in range(row):
    df_s=df.iloc[r, :]
    pref=df_s[2:].dropna().to_list()
    if fac in pref:
        cnt_number+=1
        cnt_rank+=pref.index(fac)
        length_pref+=len(pref)
length_mean=length_pref/cnt_number
print(length_mean)
print(cnt_rank/cnt_number, ((cnt_rank/cnt_number+1)/length_mean))
                
    

7.075757575757576
0.0 0.14132762312633831


In [9]:
import numpy as np

test=np.array([1.0, 2.0, 3.0])
test2=np.array([1.0, 2.0 ,3.0])
print(test+test2)

[2. 4. 6.]


##
一人当たりの効用値

In [12]:
import pandas as pd
MQ_mir=pd.read_csv(f"MultiThread/minority_acceptance_MQ0.csv",header=None , index_col=0)
utilityDict=dict()
utilityDictSum=dict()
for fac in MQ_mir.index.values.tolist():
    utilityDict[fac]=np.array([0,0,0,0])
    utilityDictSum[fac]=np.array([0,0,0,0])
n=0
trial=1000
for n in range(trial):
    utility=pd.read_csv(f"MultiThread/FacultyUtility{n}.csv",header=0, index_col=0)
    MQ_mir=pd.read_csv(f"MultiThread/minority_acceptance_MQ{n}.csv",header=None , index_col=0)
    MQ_maj=pd.read_csv(f"MultiThread/majority_acceptance_MQ{n}.csv",header=None , index_col=0)
    MR_mir=pd.read_csv(f"MultiThread/minority_acceptance_MR{n}.csv",header=None , index_col=0)
    MR_maj=pd.read_csv(f"MultiThread/majority_acceptance_MR{n}.csv",header=None , index_col=0)
    for fac in MQ_mir.index.values.tolist():
        utilityDictSum[fac]=utilityDictSum[fac]+utility.loc[fac, :]
        temp=[]
        for data in [MQ_mir, MQ_maj, MR_mir, MR_maj]:
            length=len(data.loc[fac,:].dropna().to_list())
            temp.append(length)
        temp=np.array(temp)
        utility.loc[fac, :]=(utility.loc[fac, :]/temp).fillna(0)
        utilityDict[fac]=utilityDict[fac]+utility.loc[fac, :]
for fac in MQ_mir.index.values.tolist():
    utilityDict[fac]=utilityDict[fac]/trial
    utilityDictSum[fac]=utilityDictSum[fac]/trial
utilitySumDF=pd.DataFrame(utilityDictSum)
utilityDF=pd.DataFrame(utilityDict)

NameError: name 'np' is not defined

In [191]:
utilityDF2=utilityDF[:]
print(utilityDF2)

                            法学部       経済学部   A群（思想文化）  B群（歴史文化_日本史学）  \
UtilityOfMaq(shitei)  74.514229  78.820724  73.324422      69.142418   
UtilityOfMaq(other)   64.329604  80.542990  69.285455      71.549458   
UtilityOfmir(shitei)  74.663769  78.771012  73.342538      69.076190   
UtilityOfmir(other)   64.793203  80.472300  69.073254      68.704900   

                      C群（歴史文化_東洋史学）  D群（歴史文化_西洋史学）  E群（歴史文化_考古学）  \
UtilityOfMaq(shitei)      69.366105      73.142750     64.901660   
UtilityOfMaq(other)       67.747112      72.498821     70.237028   
UtilityOfmir(shitei)      69.464258      73.132264     64.858563   
UtilityOfmir(other)       67.410799      71.130628     66.395095   

                      F群（歴史文化_美術史学）   G群（言語文化）    H群（心理学）  ...         数学  \
UtilityOfMaq(shitei)      69.435277  71.572601  91.858984  ...   0.000000   
UtilityOfMaq(other)       72.662242  68.689930  88.133239  ...  67.648522   
UtilityOfmir(shitei)      69.564261  71.687068  91.858920  ...   0.

In [192]:
utilityDF2.index=["MQの指定科類枠", "MQの全科類枠", "MRの指定科類枠", "MRの全科類枠"]
utilityDF2=utilityDF2.reindex(["MQの指定科類枠","MRの指定科類枠", "MQの全科類枠", "MRの全科類枠"])
utilityDF2=utilityDF2.T
print(utilityDF2)
utilityDF2.to_csv("生徒一人当たりの効用値.csv", encoding="utf-8-sig")
utilitySumDF.index=["MQの指定科類枠", "MQの全科類枠", "MRの指定科類枠", "MRの全科類枠"]
utilitySumDF=utilitySumDF.reindex(["MQの指定科類枠","MRの指定科類枠", "MQの全科類枠", "MRの全科類枠"])
utilitySumDF=utilitySumDF.T
print(utilitySumDF)
utilitySumDF.to_csv("全体の効用値.csv", encoding="utf-8-sig")

                MQの指定科類枠   MRの指定科類枠    MQの全科類枠    MRの全科類枠
法学部            74.514229  74.663769  64.329604  64.793203
経済学部           78.820724  78.771012  80.542990  80.472300
A群（思想文化）       73.324422  73.342538  69.285455  69.073254
B群（歴史文化_日本史学）  69.142418  69.076190  71.549458  68.704900
C群（歴史文化_東洋史学）  69.366105  69.464258  67.747112  67.410799
...                  ...        ...        ...        ...
地球惑星環境学        78.806193  78.668886  65.392911  64.231170
化学             70.756783  70.671034   0.000000   0.000000
生物化学           75.882610  75.618418   0.000000   0.000000
生物情報科学         79.638398  79.458417   0.000000   0.000000
生物学             0.000000   0.000000  69.643944  69.968681

[78 rows x 4 columns]
                  MQの指定科類枠     MRの指定科類枠      MQの全科類枠      MRの全科類枠
法学部            6437.160851  6362.760890  1146.098990  1080.164899
経済学部           7015.044444  7010.620085  1449.773821  1448.501408
A群（思想文化）        592.780378   589.307812   778.039990   769.863503
B群（歴史文化_日本史学）   2

In [2]:
import pandas as pd
MQ_mir=pd.read_csv(f"MultiThread/minority_acceptance_MQ0.csv",header=None , index_col=0)
効用増減dict=dict()
for fac in MQ_mir.index.values.tolist():
    効用増減dict[fac]=np.array([0.0,0.0,0.0,0.0])
n=0
trial=1000
for n in range(trial):
    utility=pd.read_csv(f"MultiThread/utilityDict{n}.csv",header=0)
    utility=dict(zip(utility["student"].to_list(), utility["utility"].to_list()))
    MQ_mir=pd.read_csv(f"MultiThread/minority_acceptance_MQ{n}.csv",header=None , index_col=0)
    MQ_maj=pd.read_csv(f"MultiThread/majority_acceptance_MQ{n}.csv",header=None , index_col=0)
    MR_mir=pd.read_csv(f"MultiThread/minority_acceptance_MR{n}.csv",header=None , index_col=0)
    MR_maj=pd.read_csv(f"MultiThread/majority_acceptance_MR{n}.csv",header=None , index_col=0)
    for fac in MQ_mir.index.values.tolist():
        MQ_mir_set=set(MQ_mir.loc[fac, :].dropna().to_list())
        MR_mir_set=set(MR_mir.loc[fac, :].dropna().to_list())
        kyotu=MQ_mir_set & MR_mir_set
        MQ_mir_ori=list(MQ_mir_set-kyotu)
        MR_mir_ori=list(MR_mir_set-kyotu)
        効用減mir=0
        効用増mir=0
        for student in MQ_mir_ori:
            効用減mir+=utility[student]
        for student in MR_mir_ori:
            効用増mir+=utility[student]
        MQ_maj_set=set(MQ_maj.loc[fac, :].dropna().to_list())
        MR_maj_set=set(MR_maj.loc[fac, :].dropna().to_list())
        kyotu=MQ_maj_set & MR_maj_set
        MQ_maj_ori=list(MQ_maj_set-kyotu)
        MR_maj_ori=list(MR_maj_set-kyotu)
        効用減maj=0
        効用増maj=0
        for student in MQ_maj_ori:
            効用減maj+=utility[student]
        for student in MR_maj_ori:
            効用増maj+=utility[student]
        # if len(MQ_mir_ori)!=0:
        #     効用減mir=効用減mir/len(MQ_mir_ori)
        # if len(MR_mir_ori)!=0:
        #     効用増mir=効用増mir/len(MR_mir_ori)
        # if len(MQ_maj_ori)!=0:
        #     効用減maj=効用減maj/len(MQ_maj_ori)
        # if len(MR_maj_ori)!=0:
        #     効用増maj=効用増maj/len(MR_maj_ori)
        temp=[効用減mir, 効用増mir, 効用減maj, 効用増maj]
        効用増減dict[fac]+=np.array(temp)
効用増減DF=pd.DataFrame(効用増減dict)/trial
効用増減DF=効用増減DF.T
効用増減DF.columns=["指定科類枠の効用減","指定科類枠の効用増","全科類枠の効用減","全科類枠の効用増"]
print(効用増減DF)
効用増減DF.to_csv("新制度移行による効用増減.csv", encoding="utf-8-sig")

                 指定科類枠の効用減   指定科類枠の効用増    全科類枠の効用減     全科類枠の効用増
法学部             298.064221   44.023621  320.772185   222.257085
経済学部           1275.598232   45.725636  269.425333  1267.407473
A群（思想文化）         31.333061    1.117961  105.298973   395.981073
B群（歴史文化_日本史学）    13.379779    0.000000   22.686879   390.005786
C群（歴史文化_東洋史学）     2.878316    0.000000   36.952725   650.835909
...                    ...         ...         ...          ...
地球惑星環境学           3.428616    3.361277    8.509972     8.374911
化学              116.365813  110.633488    0.000000     0.000000
生物化学             78.407763   75.421205    0.000000     0.000000
生物情報科学           39.842219   38.597836    0.000000     0.000000
生物学               0.000000    0.000000  101.527532    95.657571

[78 rows x 4 columns]


In [101]:
from import_data import facultyData
MRmaqdf=pd.read_csv("MultiThread/majority_acceptance_MR0.csv",header=None, index_col=0)
MQmaqdf=pd.read_csv("MultiThread/majority_acceptance_MQ0.csv",header=None, index_col=0)
minorityReserveB, minorityReserveDA, M, shiteiDict, fac, bunri, categoryToList=facultyData.main()
new_comer_list=list()
for fac in MRmaqdf.index.values.tolist():
    print(fac, minorityReserveDA[fac]["123456"])
    MQstudent=MQmaqdf.loc[fac,:].dropna().tolist()
    MRstudent=MRmaqdf.loc[fac,:].dropna().tolist()
    if minorityReserveDA[fac]["123456"] == len(MQstudent):
        new_comer=set(MRstudent) - set(MQstudent)
        if len(new_comer)!=0:
            print(fac, new_comer)
            for i in new_comer:
                new_comer_list.append(i)

法学部 35
法学部 {'student77'}
経済学部 18
A群（思想文化） 12
A群（思想文化） {'student486'}
B群（歴史文化_日本史学） 4
B群（歴史文化_日本史学） {'student900', 'student516', 'student245'}
C群（歴史文化_東洋史学） 6
D群（歴史文化_西洋史学） 5
E群（歴史文化_考古学） 3
F群（歴史文化_美術史学） 2
G群（言語文化） 15
H群（心理学） 4
I群（社会心理学） 3
J群（社会学） 7
基礎教育学 3
比較教育社会学 2
比較教育社会学 {'student566'}
教育実践・政策学 4
教育心理学 1
教育心理学 {'student596'}
身体教育学 10
薬学部 2
薬学部 {'student26'}
医学 1
健康総合科学 20
超域文化科学 1
地域文化研究 2
地域文化研究 {'student259'}
総合社会科学 1
総合社会科学 {'student431', 'student452', 'student970', 'student474'}
数理自然科学 0
物質基礎科学 0
統合生命科学 1
認知行動科学 0
Ａ 群 0
Ｂ 群 0
Ｂ 群 {'student298'}
教養学科国際日本研究コース 4
学際科学科国際環境学コース 4
生命化学・工学 0
応用生物学 0
森林生物科学 4
水圏生物科学 0
動物生命システム科学 0
生物素材化学 0
緑地環境学 0
森林環境資源科学 10
木質構造科学 0
生物・環境工学 12
農業・資源経済学 0
フィールド科学 1
国際開発農学 5
獣医学課程獣医学 0
社会基盤学Ａ（設計・技術戦略） 0
社会基盤学Ｂ（政策・計画）社会基盤学Ｃ 0
社会基盤学Ｃ（国際プロジェクト） 0
建築学 2
都市環境工学（環境共生・国際公共衛生・水・環境ﾊﾞｲｵ） 2
都市計画（都市と地域の分析・計画・ﾃﾞｻﾞｲﾝ） 0
機械工学A（ﾃﾞｻﾞｲﾝ・ｴﾈﾙｷﾞｰ・ﾀﾞｲﾅﾐｸｽ） 0
機械工学B 0
航空宇宙学 0
精密工学 3
電子情報工学 10
電気電子工学 0
応用物理・物理工学 3
計数工学・数理/ｼｽﾃﾑ情報 3
マテリアル工学A 2
マテリアル工学B 0
マテリアル工学B {'student463', 

In [102]:
minorityReserveB, minorityReserveDA, M, shiteiDict, fac, bunri, categoryToList=facultyData.main()
minorityDict=dict()
minorityReserveDA_sorted=dict()
for f in fac:
    minorityReserveDA_sorted[f]=sorted(minorityReserveDA[f].items(), key=lambda x:x[1], reverse=True)
    categoryToList["123456"]=[]
for f in fac:
    tmp=[]
    for _ in range(len(minorityReserveDA_sorted[f])):
        if (len(minorityReserveDA_sorted[f][_][0]) != 6):
            value=minorityReserveDA_sorted[f][_][0]
            if (categoryToList[value]!=[]) and (minorityReserveDA_sorted[f][_][1]!=0):
                tmp=tmp+categoryToList[value]
    minorityDict[f]=tmp

print(minorityDict)

{'法学部': ['l1', 'l2', 'l3', 's1', 's2', 's3'], '経済学部': ['l2', 's1', 's2', 's3'], 'A群（思想文化）': ['l3'], 'B群（歴史文化_日本史学）': ['l3'], 'C群（歴史文化_東洋史学）': ['l3'], 'D群（歴史文化_西洋史学）': ['l3'], 'E群（歴史文化_考古学）': ['l3'], 'F群（歴史文化_美術史学）': ['l3'], 'G群（言語文化）': ['l3'], 'H群（心理学）': ['l3'], 'I群（社会心理学）': ['l3'], 'J群（社会学）': ['l3'], '基礎教育学': ['l3'], '比較教育社会学': ['l3'], '教育実践・政策学': ['l3'], '教育心理学': ['l3', 's1', 's2', 's3'], '身体教育学': [], '薬学部': ['s1', 's2', 's3'], '医学': ['s3', 's2'], '健康総合科学': ['s1', 's2', 's3'], '超域文化科学': ['l3', 'l1', 'l2'], '地域文化研究': ['l3', 'l1', 'l2'], '総合社会科学': ['l1', 'l2', 'l3'], '数理自然科学': ['s1', 's2', 's3'], '物質基礎科学': ['s1', 's2', 's3'], '統合生命科学': ['s1', 's2', 's3'], '認知行動科学': ['s1', 's2', 's3', 'l1', 'l2', 'l3'], 'Ａ 群': ['l1', 'l2', 'l3', 's1', 's2', 's3'], 'Ｂ 群': ['s1', 's2', 's3', 'l1', 'l2', 'l3'], '教養学科国際日本研究コース': [], '学際科学科国際環境学コース': [], '生命化学・工学': ['s1', 's2', 's3'], '応用生物学': ['s1', 's2', 's3'], '森林生物科学': ['s1', 's2', 's3'], '水圏生物科学': ['s1', 's2', 's3'], '動物生命システム科学': ['s1', 's2', 's3'], '生

In [138]:
MRmirDF=pd.read_csv("MultiThread/minority_acceptance_MR0.csv",header=None, index_col=0)
MQmirDF=pd.read_csv("MultiThread/minority_acceptance_MQ0.csv",header=None, index_col=0)
fill_list=[]
for f in fac:
    if len(MRmirDF.loc[f,:].dropna().to_list())==len(MQmirDF.loc[f,:].dropna().to_list()):
        print(f)
        fill_list.append(f)
compareIndex=pd.read_csv("MultiThread/CompareIndex0.csv",header=0, index_col=0)
studentPref=pd.read_csv("MultiThread/created_submit_komaba0.csv",header=0, index_col=0)
row, col=studentPref.shape
leave=col
compareDict=compareIndex.to_dict(orient="index")
df_list=[]
for new_comer in new_comer_list:
    mirIndex=compareDict[new_comer]["mirIndex"]
    MaqIndex=compareDict[new_comer]["MaqIndex"]
    if MaqIndex==1:
        print(f"{new_comer}は好例？")
    new_comer_pref=studentPref.loc[new_comer, :].dropna().to_list()
    karui=new_comer_pref[0]
    new_comer_pref=new_comer_pref[1:]
    mirResult=new_comer_pref[mirIndex]
    if MaqIndex!=col:
        MaqResult=new_comer_pref[MaqIndex]
    else:
        MaqResult="留年"
    df_list.append([new_comer, karui, mirResult, MaqResult])
df=pd.DataFrame(df_list, columns=["student", "科類", "MRの学部", "MQの学部"])
df.set_index("student", inplace=True)
for student in new_comer_list:
    MQ_result=df.loc[student, "MQの学部"]
    MR_result=df.loc[student, "MRの学部"]
    karui=df.loc[student, "科類"]
    if MQ_result!="留年":
        MQmirDF_series=MQmirDF.loc[MQ_result,:].dropna()
        result=MQmirDF_series.str.contains(student)
        result=MQmirDF_series[result]
        if len(result)!=0:
            print(f"{karui}の生徒{student}は「{MQ_result}」の指定科類枠から「{MR_result}」へ")
            if MQ_result in fill_list:
                print("さらに、このケースでは必ず人員が補充されている")
        else:
            print(f"{karui}の生徒{student}は「{MQ_result}」の全科類枠から「{MR_result}」へ")

法学部
経済学部
A群（思想文化）
B群（歴史文化_日本史学）
C群（歴史文化_東洋史学）
D群（歴史文化_西洋史学）
E群（歴史文化_考古学）
H群（心理学）
I群（社会心理学）
J群（社会学）
基礎教育学
比較教育社会学
教育実践・政策学
教育心理学
身体教育学
薬学部
医学
超域文化科学
数理自然科学
物質基礎科学
統合生命科学
認知行動科学
Ａ 群
Ｂ 群
教養学科国際日本研究コース
学際科学科国際環境学コース
応用生物学
森林生物科学
水圏生物科学
動物生命システム科学
生物素材化学
緑地環境学
森林環境資源科学
木質構造科学
生物・環境工学
農業・資源経済学
フィールド科学
国際開発農学
獣医学課程獣医学
社会基盤学Ａ（設計・技術戦略）
社会基盤学Ｂ（政策・計画）社会基盤学Ｃ
社会基盤学Ｃ（国際プロジェクト）
建築学
都市環境工学（環境共生・国際公共衛生・水・環境ﾊﾞｲｵ）
都市計画（都市と地域の分析・計画・ﾃﾞｻﾞｲﾝ）
機械工学A（ﾃﾞｻﾞｲﾝ・ｴﾈﾙｷﾞｰ・ﾀﾞｲﾅﾐｸｽ）
機械工学B
精密工学
電子情報工学
電気電子工学
応用物理・物理工学
計数工学・数理/ｼｽﾃﾑ情報
マテリアル工学A
マテリアル工学B
マテリアル工学C (ﾅﾉ・機能ﾏﾃﾘｱﾙ）
化学システム工学
システム創成A
システム創成B
システム創成C
数学
情報科学
物理学
天文学
地球惑星物理学
地球惑星環境学
生物化学
生物情報科学
生物学
student452は好例？
student229は好例？
l2の生徒student486は「教育実践・政策学」の全科類枠から「A群（思想文化）」へ
l2の生徒student900は「総合社会科学」の指定科類枠から「B群（歴史文化_日本史学）」へ
s1の生徒student516は「応用化学」の指定科類枠から「B群（歴史文化_日本史学）」へ
s2の生徒student245は「法学部」の全科類枠から「B群（歴史文化_日本史学）」へ
s1の生徒student566は「生命化学・工学」の指定科類枠から「比較教育社会学」へ
l2の生徒student596は「学際科学科国際環境学コース」の全科類枠から「教育心理学」へ
s1の生徒student26は「航空宇宙学」の指定科類枠から「薬学部」へ
s1の生徒student431は「航空宇宙学」の指定科類枠から「

In [3]:
import copy
from import_data import facultyData
minorityReserveB, minorityReserveDA, M, shiteiDict, fac, bunri, categoryToList=facultyData.main()
n=0
trial=1000
fac_init=dict()
for f in fac:
    fac_init[f]=0
submitDict=dict()
for k in ["l1", "l2", "l3","s1","s2", "s3"]:
    fac_dict=copy.deepcopy(fac_init)
    submitDict[k]=fac_dict
for n in range(trial):
    submit=pd.read_csv(f"MultiThread/created_submit_komaba{n}.csv",header=0,index_col=0)
    index=submit.index.values.tolist()
    for student in index:
        sData=submit.loc[student,:].dropna().to_list()
        karui=sData[0]
        pref=sData[1:]
        karui_submit=submitDict[karui]
        for p in pref:
            karui_submit[p]+=1
        
for k in ["l1", "l2", "l3","s1","s2", "s3"]:
    for f in fac:
        submitDict[k][f]=submitDict[k][f]/trial

In [4]:
df=pd.DataFrame.from_dict(submitDict)
print(df)
df.to_csv("Result/提出平均値.csv", encoding="utf-8-sig")

                    l1       l2      l3       s1      s2     s3
法学部            103.626   35.472  47.440   32.246  25.496  0.006
経済学部            33.589  132.000  40.235  110.090  70.876  0.004
A群（思想文化）        18.091   42.147  49.065   21.346  19.208  0.841
B群（歴史文化_日本史学）   12.719   31.519  29.354    5.794   4.978  0.007
C群（歴史文化_東洋史学）   10.917   35.628  30.903    5.291   3.229  0.000
...                ...      ...     ...      ...     ...    ...
地球惑星環境学          0.003    0.002   0.002   38.412  26.364  0.006
化学               0.002    0.002   0.001   41.021  38.314  0.004
生物化学             0.003    0.004   0.003   16.524  38.796  0.005
生物情報科学           0.001    0.003   0.000   39.120  34.021  0.000
生物学              0.005    0.005   0.006   19.305  32.828  0.001

[78 rows x 6 columns]


In [ ]:
n=0
trial=1000
fac_init=dict()
for f in fac:
    fac_init[f]=0
naiteiDict=dict()
for k in ["l1", "l2", "l3","s1","s2", "s3"]:
    fac_dict=copy.deepcopy(fac_init)
    naiteiDict[k]=fac_dict
for n in range(trial):
    submit=pd.read_csv(f"MultiThread/submitMaqResultKomaba{n}.csv",header=0,index_col=0)
    index=submit.index.values.tolist()
    for student in index: